# Maui AirBnB 🌴🏡🌴

## What Questions are we asking?
- what can we learn about different hosts and areas?
- what can we learn from predictions? (ex. locations, price, reviews, etc.)
- which hosts are the busiest and why? 
- is there any noticable different of traffice among different areas and what could be the reason for it? 

## Call in the Libraries 📝

In [1]:
# 1 - Data Manipulation
import pandas as pd
import numpy as np
import re

# 2 - Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

## Call in the Datasets 📃

In [19]:
listings_data = pd.read_csv('../raw_data/h_listings.csv')
calendar_data = pd.read_csv('../raw_data/h_calendar.csv')
review_data = pd.read_csv('../raw_data/h_reviews.csv')

In [21]:
listings_data.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,81566,https://www.airbnb.com/rooms/81566,20230610213045,2023-06-11,city scrape,Rental unit in Haleiwa · ★4.67 · 2 bedrooms · ...,2 bedroom/1 bath ocean view & beach access<br ...,There are so many awesome things to do on the ...,https://a0.muscache.com/pictures/555752/35365a...,442490,...,4.75,4.86,4.62,ta-020-657-9712-01,f,1,1,0,0,1.76
1,81582,https://www.airbnb.com/rooms/81582,20230610213045,2023-06-11,city scrape,Home in Pāhoa · ★4.94 · 2 bedrooms · 3 beds · ...,Quaint retreat COTTAGE; entire 2 bedroom 1 ba...,This is a quaint neighborhood nestled in the O...,https://a0.muscache.com/pictures/555927/be245b...,442698,...,4.94,4.94,4.83,STVR-19-355130,f,2,2,0,0,1.27
2,83221,https://www.airbnb.com/rooms/83221,20230610213045,2023-06-11,city scrape,Cabin in Pāhoa · ★4.93 · 1 bedroom · 1 bed · 1...,The Enchanting Rainforest Hideaway is a very ...,NaN,https://a0.muscache.com/pictures/568427/f7ce56...,451536,...,4.96,4.86,4.94,STVR-19-364512,f,1,1,0,0,2.11


In [12]:
calendar_data.tail()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
11897907,908851517358534174,2024-06-05,f,$167.00,$167.00,2.0,1125.0
11897908,908851517358534174,2024-06-06,t,$167.00,$167.00,2.0,1125.0
11897909,908851517358534174,2024-06-07,t,$167.00,$167.00,2.0,1125.0
11897910,908851517358534174,2024-06-08,t,$167.00,$167.00,2.0,1125.0
11897911,908851517358534174,2024-06-09,t,$167.00,$167.00,2.0,1125.0


In [11]:
review_data.tail()

,listing_id,id,date,reviewer_id,reviewer_name,comments
1085783,907248461538337553,587773101242885139,2022-03-21,87653080,Diana,"Great location, surf right out front, walk to ..."
1085784,907249057198692113,595070396950612693,2022-03-31,88238338,Neli,We had a wonderful stay here with my family! W...
1085785,907249057198692113,758819324310468241,2022-11-12,361671022,Sean,"The unit was great. Nice location, close to th..."
1085786,907249057198692113,794374596374440225,2022-12-31,77588134,Rajnish,My family and I had an amazing time at the pro...
1085787,907253425303087407,658900002099066881,2022-06-27,384456068,Jerrold,Excellent location. Great view of sunset and ...


In [23]:
# looking through the column names to find what is needed.
column_names = listings_data.columns.tolist()
column_names

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'source',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30

In [31]:
# Made an alternate Data set of just the essential columns

listings_essential = listings_data[[
    'id',
    'name',
    'host_id',
    'host_name',
    'neighbourhood_cleansed',
    'neighbourhood_group_cleansed',
    'room_type',  
    'price',
    'minimum_nights',
    'maximum_nights',
    'number_of_reviews',
    'calculated_host_listings_count',
    'availability_365'
    ]]

In [32]:
#seeing what the data set looks like
listings_essential.head(3)

,id,name,host_id,host_name,neighbourhood_cleansed,neighbourhood_group_cleansed,room_type,price,minimum_nights,maximum_nights,number_of_reviews,calculated_host_listings_count,availability_365
0,81566,Rental unit in Haleiwa · ★4.67 · 2 bedrooms · ...,442490,Susan,North Shore Oahu,Honolulu,Entire home/apt,$250.00,4,1125,260,1,249
1,81582,Home in Pāhoa · ★4.94 · 2 bedrooms · 3 beds · ...,442698,Elizabeth,Puna,Hawaii,Entire home/apt,$119.00,2,10,181,2,69
2,83221,Cabin in Pāhoa · ★4.93 · 1 bedroom · 1 bed · 1...,451536,Connie And Andrew,Puna,Hawaii,Entire home/apt,$117.00,3,30,311,1,323


## Data Exploration and Manipulation ⛏

In [35]:
listings_essential.isna().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_cleansed            0
neighbourhood_group_cleansed      0
room_type                         0
price                             0
minimum_nights                    0
maximum_nights                    0
number_of_reviews                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [34]:
#filling in the host name with "unknown"
listings_essential['host_name'].fillna('Unknown', inplace = True)

/tmp/ipykernel_5676/95726933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_essential['host_name'].fillna('Unknown', inplace = True)


In [39]:
listings_essential.shape

(32597, 13)

In [41]:
listings_essential.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'room_type', 'price', 'minimum_nights',
       'maximum_nights', 'number_of_reviews', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')